### Code for running experiment
### Callin Switzer
### 21 Feb 2019

In [ ]:
import nectarUtils
from nectarUtils import *
import nectarUtils as nu

import importlib


%matplotlib inline
print(sys.version)
print(sys.executable)

# define directories
baseDir = os.getcwd()

# may want to make this directory somewhere else, if dropbox becomes a problem
dataDir = r"D:\Dropbox\AcademiaDropbox\UW\BeeDecisionProject\NectarData"
if not os.path.isdir(dataDir):
    os.mkdir(dataDir)

figDir = r"D:\Dropbox\AcademiaDropbox\UW\BeeDecisionProject\NectarFigs"
if not os.path.isdir(figDir):
    os.mkdir(figDir)

In [ ]:
# list serial ports
nu.serial_ports()

In [ ]:
# connect to com8
PORT1 = "COM4"
connected1 = False
if "ser1" in globals():
    ser1.close()
ser1 = serial.Serial(PORT1,9600, timeout=1.0) # stop if no data comes in 1 second
while not connected1:
    serin1 = ser1.read()
    connected1 = True
    print("connected to arduino on " + PORT1)
str(ser1.readline().decode("UTF-8"))    

In [ ]:
ser1.write("ff".encode("utf-8"))

In [ ]:
#ser1.close()

In [ ]:
# reload package if changed
_ = importlib.reload(nectarUtils)

calb = nu.calibrate(ser1)

In [ ]:
# reload package if changed
_ = importlib.reload(nectarUtils)


# read and save data
stt = time.time()

newDat = nu.readAndSave(ser1, maxTime=600*2, saveData=True, 
                        dataDir = dataDir, timeout = 600*2, 
                        minRewardThreshold = int(1.10*calb["topBaseline"]), 
                       colNames = calb["colNames"], 
                       baseSensorThreshold = calb['base_dec_bound'], 
                       calibrationInfo = calb)

print(time.time() - stt)
newDat.head()

newDat['timestamp'] = pd.to_datetime(newDat['timestamp'])
newDat['delta'] = (newDat['timestamp']-newDat['timestamp'].shift()).fillna(pd.Timedelta(seconds=0))

newDat.plot(y=['top', 'mid', 'base'], x = "timestamp", style='-', figsize=np.array([15, 5]))

plt.scatter(y=newDat['top'], x = newDat["timestamp"])
plt.vlines(newDat[newDat.notes == "reward Triggered"]["timestamp"], ymin = 0, ymax = 1000, label = "reward")
plt.show()
#newDat.plot(y=['delta'], x = "timestamp", style='-')

In [1]:
#!/usr/bin/env python
import subprocess
# #  
# # Call a bash script
# subprocess.call(['./myBashScript.sh'])
 
# # Call a javascript script with node
# subprocess.call(['node', './myJSScript.js'])

In [ ]:
# reload package if changed
_ = importlib.reload(nectarUtils)


# read and save data
stt = time.time()

newDat = nu.readOnly(ser1, maxTime=5, saveData=True, dataDir = dataDir, timeout = 600*2)

print(time.time() - stt)
newDat.head()

newDat['timestamp'] = pd.to_datetime(newDat['timestamp'])
newDat['delta'] = (newDat['timestamp']-newDat['timestamp'].shift()).fillna(pd.Timedelta(seconds=0))

newDat.plot(y=['top_sensor', 'mid_sensor', 'base_sensor'], x = "timestamp", style='-', figsize=np.array([15, 5]))

plt.scatter(y=newDat['top_sensor'], x = newDat["timestamp"])
plt.vlines(newDat[newDat.notes == "reward Triggered"]["timestamp"], ymin = 0, ymax = 1000, label = "reward")

#newDat.plot(y=['delta'], x = "timestamp", style='-')

In [ ]:
newDat.plot(y=['top_sensor', 'mid_sensor', 'base_sensor'], x = "timestamp", style='-', figsize=np.array([15, 5]))

plt.scatter(y=newDat['top_sensor'], x = newDat["timestamp"])
plt.vlines(newDat[newDat.notes == "reward Triggered"]["timestamp"], ymin = 0, ymax = 1000, label = "reward")
plt.hlines(y = 150, xmin = np.min(newDat["timestamp"]), xmax = np.max(newDat["timestamp"]), label = "thresh")

In [ ]:
int.from_bytes(b'b', byteorder='big')

In [ ]:
numSteps = 4
[ser1.write("b".encode("utf-8")) for ii in range(numSteps)]


# for com7
df1[["base_sensor", "mid_sensor", 
     "top_sensor", "limit_1", "limit_2"]] = \
    df1[["base_sensor", "mid_sensor", 
        "top_sensor", "limit_1", "limit_2"]].astype(int)

df1.head()

In [ ]:
# for com8, switch base and mid refref, double check
tt = readData(ser1, readlen=10, wait_time=0.0, save=True, returnVals = True)
df1 = pd.DataFrame(tt, columns=["base_sensor", "mid_sensor", "top_sensor", "limit_1", "limit_2", "timestamp"])

df1[["mid_sensor", "base_sensor",
     "top_sensor", "limit_1", "limit_2"]] = \
    df1[["base_sensor", "mid_sensor", 
        "top_sensor", "limit_1", "limit_2"]].astype(int)

print(df1.shape)
df1.head()

In [ ]:
df1.tail()

In [ ]:
#np.array(df1.iloc[:,0].astype(int))

In [ ]:
ax1 = df1.iloc[:, 0:3].plot()
lines, labels = ax1.get_legend_handles_labels()
ax1.legend(lines, labels, loc='best')  

In [ ]:
plt.plot(df1[["base_sensor", "mid_sensor", 
     "top_sensor"]])
plt.show()

In [ ]:
plt.plot(np.array(df1.iloc[:,1].astype(int)))

In [ ]:
plt.plot(np.array(df1.iloc[:,2].astype(int)))

In [ ]:
(tt[0, 5])

In [ ]:
# for com7
# top sensor
tt[:, 2]

# mid sensor
tt[:, 1]

# base sensor
tt[:, 0]

In [ ]:
plt.plot(tt[:,2])

In [ ]:
plt.plot(tt)

In [ ]:
#ser1.close()

In [ ]:
def moveToTop(serial_con, cutoff = 650):
    # refref: may want to go 1 or two more moves forward after cutoff is passed --- 
    ## the cutoff is the meniscus
    [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:, [1,3,4]]
    print(topVal)
    while (topVal > cutoff) and not topLim:
        #move forward
        ser1.write("f".encode("utf-8"))
        
        # read data again
        [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:,[1,3,4]]
        # rror if limit switch is hit
        if topLim:
            raise RuntimeError('Hit upper limit switch')
        
        
        
        

In [ ]:
# refref: problem -- liquid stays stuck on the sides -- may need to move back slower

def moveBack(serial_con, cutoff = 650):
    
    [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:, [1,3,4]]
    
    
    while (topVal < cutoff) and not bottomLim:
        #move backward
        for jj in range(7):
            ser1.write("b".encode("utf-8"))
            time.sleep(0.3)
        
        # read data again
        [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:, [1,3,4]]
        # rror if limit switch is hit
        if bottomLim:
            raise RuntimeError('Hit lower limit switch')

In [ ]:
readData(ser1, 1, 0)

In [ ]:
moveToTop(ser1)
readData(ser1, 1, 0)

In [ ]:
moveBack(ser1)
readData(ser1, 1, 0)

In [ ]:
ser1.write("f".encode("utf-8"))

In [ ]:
[[vals, bottomLim, topLim]] = readData(ser1, 1, 0)[:, 2:]

In [ ]:
readData(ser1, 1, 0)[:, 2:]

In [ ]:
toplim = 1

In [ ]:
not toplim

for ii in range(20): 
    written = ser1.write("f".encode("utf-8"))


ser1.write("f".encode("utf-8"))

for ii in range(100): 
    written = ser1.write("b".encode("utf-8"))

"c".encode("utf-8")
int.from_bytes(b'c', byteorder='big') # this is what the arduino will see

int.from_bytes(b'c', byteorder='big')

ser1.write("c".encode("utf-8"))